#### Extracts outcomes from MIMIC-IV EHR data and links clinical notes from previous inpatient episodes.

In [ ]:
%pip install statannotations missingno
!python.exe -m pip install --upgrade pip

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from datetime import timedelta, datetime
from tqdm import tqdm
import numpy as np
from scipy import stats, special

import os
import json
import pprint
import missingno as msno
from statannotations.Annotator import Annotator
import warnings

#### Load raw MIMIC-IV files

In [ ]:
### Helper-functions for extracting EHR data
def dataframe_from_csv(path, compression='gzip', header=0, index_col=0, chunksize=None):
    return pd.read_csv(path, compression=compression, header=header, index_col=index_col, chunksize=None)

def read_admissions_table(mimic4_path):
    admits = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/admissions.csv.gz'))
    admits = admits.reset_index()
    admits = admits[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime',
                     'race', 'marital_status', 'insurance', 'admission_location', 'discharge_location']]
    admits.admittime = pd.to_datetime(admits.admittime)
    admits.dischtime = pd.to_datetime(admits.dischtime)
    admits.deathtime = pd.to_datetime(admits.deathtime)
    ### Get eligible admissions with ED attendance and complete sensitive data
    print("Original number of admissions:", admits.shape[0], admits.subject_id.nunique())
    admits = admits[(admits.edregtime.notnull()) & (admits.edouttime.notnull())]
    admits = admits[(admits.marital_status.notnull()) & (admits.race.notnull()) & (admits.insurance.notnull())]
    print("Number of admissions with complete data:", admits.shape[0], admits.subject_id.nunique())
    ### Validate timestamps
    admits = admits[(admits.edregtime < admits.admittime)]
    admits = admits[(admits.edregtime < admits.dischtime) & (admits.edouttime < admits.dischtime)]
    admits = admits[(admits.edouttime > admits.edregtime)&(admits.admittime < admits.dischtime)]
    print("Number of admissions after timestamp validation:", admits.shape[0], admits.subject_id.nunique())
    ### Set ground-truths for prediction
    admits['los_days'] = (admits.dischtime - admits.admittime).dt.total_seconds() / (24 * 60 * 60)
    admits['ext_stay_7'] = (admits.los_days > 7).astype(int)
    #admits['in_hosp_death'] = np.where((admits.deathtime<admits.dischtime)&(admits.deathtime>admits.admittime), 1, 0).astype(int)
    ### Get final admissions as prediction target
    #admits = admits.sort_values(by=['subject_id', 'admittime']).drop_duplicates(subset='subject_id', keep='last')
    #print("Number of valid index admissions:", admits.shape[0])
    return admits

def read_patients_table(mimic4_path, adm_data):
    pats = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/patients.csv.gz'))
    pats = pats.reset_index()
    print("Original number of patients:", pats.shape[0], pats.subject_id.nunique())
    pats = pats[pats.subject_id.isin(adm_data.subject_id)]
    print("Number of patients with validated ED attendances:", pats.shape[0], pats.subject_id.nunique())
    pats = pats[['subject_id','gender', 'dod', 'anchor_age', 'anchor_year']]
    pats['yob']= pats['anchor_year'] - pats['anchor_age']
    #pats.dob = pd.to_datetime(pats.dob)
    pats.dod = pd.to_datetime(pats.dod)
    pats=pats.drop(columns=['anchor_year'])
    pats = pd.merge(pats, adm_data, on='subject_id', how='left')
    pats['in_hosp_death'] = np.where((pats.dod<pats.dischtime)&(pats.dod>pats.admittime), 1, 0).astype(int)
    pats['non_home_discharge'] = ((~pats.discharge_location.str.contains('HOME|DIED|AGAINST ADVICE', na=True))&(pats.in_hosp_death==0)).astype(int)
    return pats

def read_notes_table(mimic4_path, adm_data, ext_path=None):
    notes = dataframe_from_csv(os.path.join(mimic4_path, 'note/discharge.csv.gz'))
    notes = notes.reset_index()
    print("Original number of notes:", notes.shape[0], notes.subject_id.nunique())
    notes = notes[notes.subject_id.isin(adm_data.subject_id)]
    notes['charttime'] = pd.to_datetime(notes.charttime)
    print("Number of notes with validated ED attendances:", notes.shape[0], notes.subject_id.nunique())
    if ext_path:
        notes_prep = pd.read_csv(ext_path, sep=',', low_memory=False)
        notes_prep = notes_prep.reset_index()
        notes = pd.merge(notes, notes_prep, how='left', on='note_id')
        print("Number of total matching preprocessed notes:", notes.input.notnull().sum(), notes.target.notnull().sum())
        print("Unique patients with matching preprocessed notes:", notes[notes.input.notnull()].subject_id.nunique(), 
              notes[notes.target.notnull()].subject_id.nunique())
        
    adm_notes = pd.merge(adm_data[['subject_id', 'hadm_id', 'admittime']], 
                         notes[['note_id', 'subject_id', 'hadm_id', 'charttime', 'text', 'input', 'target',
                                      'input_tokens', 'target_tokens']], how='left', on=['subject_id', 'hadm_id'])
    adm_notes = adm_notes[(adm_notes.target.notnull())]
    ### Get last admission for each patient to discard notes from last inpatient episode
    adm_last = adm_notes.sort_values(['subject_id', 'admittime']).drop_duplicates(subset=['subject_id'], keep='last')
    adm_last = adm_last.rename(columns={'admittime':'last_admittime'})
    ### Get list of eligible notes as historical data
    adm_lkup = pd.merge(adm_notes, adm_last[['subject_id', 'last_admittime']], on='subject_id', how='left')
    adm_lkup = adm_lkup[adm_lkup.admittime < adm_lkup.last_admittime]
    adm_notes = adm_notes[adm_notes.hadm_id.isin(adm_lkup.hadm_id)]
    print("Unique patients with matching preprocessed notes:", notes[notes.input.notnull()].subject_id.nunique(), 
              notes[notes.target.notnull()].subject_id.nunique())
    print('Min, Mean and Max historical notes per patient:', adm_notes.groupby('subject_id').size().min(),
          adm_notes.groupby('subject_id').size().mean(), adm_notes.groupby('subject_id').size().max())
    return adm_notes

def read_icu_table(mimic4_path, adm_data):
    icu = dataframe_from_csv(os.path.join(mimic4_path, 'icu/icustays.csv.gz'))
    icu = icu.reset_index()
    icu = icu[['subject_id', 'hadm_id', 'intime', 'outtime', 'los']]
    print("Original number of ICU stays:", icu.shape[0], icu.subject_id.nunique())
    icu = icu[(icu.subject_id.isin(adm_data.subject_id))&(icu.hadm_id.isin(adm_data.hadm_id))]
    print("Number of ICU stays with validated ED attendances:", icu.shape[0], icu.subject_id.nunique())
    icu.intime = pd.to_datetime(icu.intime)
    icu.outtime = pd.to_datetime(icu.outtime)
    icu_eps = pd.merge(adm_data, icu, how='left', on=['subject_id', 'hadm_id']).sort_values(['subject_id', 'hadm_id', 'intime']).drop_duplicates(subset=['subject_id', 'hadm_id'], 
                                                                                                                                                 keep='last')
    icu_eps['icu_admission'] = np.where((icu_eps.intime>icu_eps.admittime)&(icu_eps.outtime<icu_eps.dischtime), 1, 0)
    return icu_eps.rename(columns={'los':'icu_los_days'})

########################## DIAGNOSES ##########################
def read_diagnoses_icd_table(mimic4_path):
    diag = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/diagnoses_icd.csv.gz'))
    diag.reset_index(inplace=True)
    return diag


def read_d_icd_diagnoses_table(mimic4_path):
    d_icd = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/d_icd_diagnoses.csv.gz'))
    d_icd.reset_index(inplace=True)
    return d_icd[['icd_code', 'long_title']]


def read_diagnoses(mimic4_path, adm_data):
    
    dg_df = read_diagnoses_icd_table(mimic4_path).merge(
        read_d_icd_diagnoses_table(mimic4_path), how='inner', left_on=['icd_code'], right_on=['icd_code']
    )
    print("Original number of diagnoses:", dg_df.shape[0], dg_df.subject_id.nunique())
    ### Get last admission for each patient and drop it to get comorbidity history
    adm_last = adm_data.sort_values(['subject_id', 'admittime']).drop_duplicates(subset=['subject_id'], keep='last')
    adm_last = adm_last.rename(columns={'admittime':'last_admittime'})
    ### Get list of eligible hospital episodes as historical data
    adm_lkup = adm_data[~adm_data.hadm_id.isin(adm_last.hadm_id)]
    adm_lkup = pd.merge(adm_lkup, adm_last[['subject_id', 'last_admittime']], on='subject_id', how='left')
    adm_lkup = adm_lkup[adm_lkup.admittime < adm_lkup.last_admittime]
    ### Filter diagnoses for lookup episodes
    dg_df = dg_df[dg_df.subject_id.isin(adm_lkup.subject_id)]
    dg_df = dg_df[dg_df.hadm_id.isin(adm_lkup.hadm_id)]
    print("Number of previous diagnoses recorded in historical ED metadata:", dg_df.shape[0], dg_df.subject_id.nunique())
    return dg_df


def standardize_icd(mapping, df, root=False):
    """Takes an ICD9 -> ICD10 mapping table and a diagnosis dataframe; adds column with converted ICD10 column"""

    def icd_9to10(icd):
        # If root is true, only map an ICD 9 -> 10 according to the ICD9's root (first 3 digits)
        if root:
            icd = icd[:3]
        try:
            # Many ICD-9's do not have a 1-to-1 mapping; get first index of mapped codes
            return mapping.loc[mapping.diagnosis_code == icd].icd10cm.iloc[0]
        except:
            print("Error on code", icd)
            return np.nan

    # Create new column with original codes as default
    col_name = 'icd10_convert'
    if root: col_name = 'root_' + col_name
    df[col_name] = df['icd_code'].values

    # Group identical ICD9 codes, then convert all ICD9 codes within a group to ICD10
    for code, group in df.loc[df.icd_version == 9].groupby(by='icd_code'):
        new_code = icd_9to10(code)
        for idx in group.index.values:
            # Modify values of original df at the indexes in the groups
            df.at[idx, col_name] = new_code

########################## LABS ##########################
def read_labevents_table(mimic4_path):
    labevents = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/labevents.csv.gz'), chunksize=1000)
    labevents.reset_index(inplace=True)
    return labevents[['subject_id', 'itemid', 'hadm_id', 'charttime', 'storetime', 'value', 'valueuom', 'flag']]


def read_d_labitems_table(mimic4_path):
    labitems = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/d_labitems.csv.gz'), chunksize=1000)
    labitems.reset_index(inplace=True)
    return labitems[['itemid', 'label', 'category', 'lonic_code']]


def read_labs(mimic4_path):
    labevents = read_labevents_table(mimic4_path)
    labitems =  read_d_labitems_table(mimic4_path)
    return labevents(mimic4_path).merge(
        labitems, how='inner', left_on=['itemid'], right_on=['itemid']
    )

########################## PROCEDURES ##########################
def read_procedures_icd_table(mimic4_path):
    proc = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/procedures_icd.csv.gz'))
    proc.reset_index(inplace=True)
    return proc


def read_d_icd_procedures_table(mimic4_path):
    p_icd = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/d_icd_procedures.csv.gz'))
    p_icd.reset_index(inplace=True)
    return p_icd[['icd_code', 'long_title']]


def read_procedures(mimic4_path):
    return read_procedures_icd_table(mimic4_path).merge(
        read_d_icd_procedures_table(mimic4_path), how='inner', left_on=['icd_code'], right_on=['icd_code']
    )

########################## MEDICATIONS ##########################
def read_prescriptions_table(mimic4_path):
    meds = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/prescriptions.csv.gz'))
    meds = meds.reset_index()
    return meds[['subject_id', 'hadm_id', 'starttime', 'stoptime', 'ndc', 'gsn', 'drug', 'drug_type']]

def get_generic_drugs(mapping, df):
    """Takes NDC product table and prescriptions dataframe; adds column with NDC table's corresponding generic name"""

    def brand_to_generic(ndc):
        # We only want the first 2 sections of the NDC code: xxxx-xxxx-xx
        matches = list(re.finditer(r"-", ndc))
        if len(matches) > 1:
            ndc = ndc[:matches[1].start()]
        try:
            return mapping.loc[mapping.PRODUCTNDC == ndc].NONPROPRIETARYNAME.iloc[0]
        except:
            print("Error: ", ndc)
            return np.nan

    df['generic_drug_name'] = df['ndc'].apply(brand_to_generic)

########################## PREPROCESSING ##########################
def read_icd_mapping(map_path):
    mapping = pd.read_csv(map_path, header=0, delimiter='\t', low_memory=False, encoding='iso-8859-1')
    mapping.diagnosis_description = mapping.diagnosis_description.apply(str.lower)
    return mapping

def preproc_icd_module(module, icd_map_path=None, map_code_colname=None, only_icd10=True, cc_dict_path=None) -> pd.DataFrame:
    """Takes an module dataset with ICD codes and puts it in long_format, optionally mapping ICD-codes by a mapping table path"""

    def standardize_icd(mapping, df, root=False):
        """Takes an ICD9 -> ICD10 mapping table and a modulenosis dataframe; adds column with converted ICD10 column"""
        
        def icd_9to10(icd):
            # If root is true, only map an ICD 9 -> 10 according to the ICD9's root (first 3 digits)
            if root:
                icd = icd[:3]
            try:
                # Many ICD-9's do not have a 1-to-1 mapping; get first index of mapped codes
                return mapping.loc[mapping[map_code_colname] == icd].icd10cm.iloc[0]
            except:
                #print("Error on code", icd)
                return np.nan

        # Create new column with original codes as default
        col_name = 'icd10_convert'
        if root: col_name = 'root_' + col_name
        df[col_name] = df['icd_code'].values

        # Group identical ICD9 codes, then convert all ICD9 codes within a group to ICD10
        for code, group in tqdm(df.loc[df.icd_version == 9].groupby(by='icd_code')):
            new_code = icd_9to10(code)
            for idx in group.index.values:
                # Modify values of original df at the indexes in the groups
                df.at[idx, col_name] = new_code

        if only_icd10:
            # Column for just the roots of the converted ICD10 column
            df['root'] = df[col_name].apply(lambda x: x[:3] if type(x) is str else np.nan)

    print(module.shape)
    print('Unique diagnoses', module['icd_code'].nunique())

    # Optional ICD mapping if argument passed
    if icd_map_path:
        icd_map = read_icd_mapping(icd_map_path)
        #print(icd_map)
        standardize_icd(icd_map, module, root=True)
        module = module[module['root_icd10_convert'].notnull()]
        print("# unique ICD-9 codes",module[module['icd_version']==9]['icd_code'].nunique())
        print("# unique ICD-10 codes",module[module['icd_version']==10]['icd_code'].nunique())
        print("# unique ICD-10 codes (After converting ICD-9 to ICD-10)",module['root_icd10_convert'].nunique())
        print("# unique ICD-10 codes (After clinical grouping ICD-10 codes)",module['root'].nunique())
        print("# Unique patients:  ", module.hadm_id.nunique())

    module = module[['subject_id','hadm_id','seq_num','long_title','root_icd10_convert']]
    #### Create features for long-term chronic conditions
    if cc_dict_path:
        with open(cc_dict_path, 'r') as json_dict:
            ltc_dict = json.load(json_dict)
        ### Initialise long-term condition column
        module['ltc_code'] = 'Undefined'
        for ltc_code, icd_codes in ltc_dict.items():
            module['ltc_code'] = np.where(
                module['root_icd10_convert'].str.startswith(tuple(icd_codes)),
                ltc_code,
                module['ltc_code']
            )

    return module

### Helper util function for physical-mental multimorbidity detection
def contains_both_ltc_types(ltc_set):
    return all(any(code.startswith(prefix) for code in ltc_set) for prefix in ['physltc_', 'menltc_'])

def prepare_unique_patient_features(adm_data, diag_data, ltc_dict_path):
    ### Get last ED episode for each patient and drop it to get historical features from the EHR
    adm_last = adm_data.sort_values(['subject_id', 'admittime']).drop_duplicates(subset=['subject_id'], keep='last')
    ### Comorbidity history
    diag_flat = diag_data[diag_data.ltc_code!='Undefined']
    print("Number of previous diagnoses recorded in historical ED metadata:", diag_data.shape[0], diag_data.subject_id.nunique())
    ### Create list for each row in ltc_code column
    diag_flat = diag_flat.groupby(['subject_id'])['ltc_code'].agg(set).reset_index()
    ### If dict is populated generate categorical columns for each long-term condition
    if ltc_dict_path:
        with open(ltc_dict_path, 'r') as json_dict:
            ltc_dict = json.load(json_dict)
        for ltc_code, _ in ltc_dict.items():
            diag_flat[ltc_code] = diag_flat['ltc_code'].apply(lambda x: 1 if ltc_code in x else 0)

    ### Create features for multimorbidity
    diag_flat['phys_men_multimorbidity'] = diag_flat['ltc_code'].apply(contains_both_ltc_types)
    diag_flat['n_unique_conditions'] = diag_flat['ltc_code'].apply(len).astype(np.int8)
    diag_flat['is_multimorbid'] = np.where((diag_flat['ltc_code'].str.len()>1), 1, 0)
    diag_flat['is_complex_multimorbid'] = np.where((diag_flat['ltc_code'].str.len()>3), 1, 0)
    ### Merge with base patient data
    adm_last = pd.merge(adm_last, diag_flat, on='subject_id', how='left')
    adm_last[diag_flat.drop(['subject_id', 'ltc_code'], axis=1).columns] = adm_last[diag_flat.drop(['subject_id', 'ltc_code'], axis=1).columns].fillna(0).astype(np.int8)
    return adm_last

def read_ndc_mapping(map_path):
    ndc_map = pd.read_csv(map_path, header=0, delimiter=',', low_memory=False, encoding='iso-8859-1')
    ndc_map.NONPROPRIETARYNAME = ndc_map.NONPROPRIETARYNAME.fillna("")
    ndc_map.NONPROPRIETARYNAME = ndc_map.NONPROPRIETARYNAME.apply(str.lower)
    ndc_map.columns = list(map(str.lower, ndc_map.columns))
    return ndc_map

##### Parse health outcomes and input features

In [ ]:
icd_path = '../../data/MIMIC-IV/config/icd9to10.txt'
ndc_path = '../../data/MIMIC-IV/config/NDC_product_table.csv'
icd10_map_path = '../../data/MIMIC-IV/config/icd10_codes.json'
notes_ext_path = '../../data/MIMIC-IV/mimic-iv-bhc.csv'
core_path = '../../data/MIMIC-IV/mimiciv/3.1'
core_path_ed = '../../data/MIMIC-IV/mimic-iv-ed/3.1'
core_path_txt = '../../data/MIMIC-IV/mimic-iv-note/3.1'

In [ ]:
adm_data = read_admissions_table(core_path)
demo_data = read_patients_table(core_path, adm_data)

In [ ]:
demo_data.shape, adm_data.shape

In [ ]:
print(demo_data.ext_stay_7.value_counts(normalize=True))
print(demo_data.in_hosp_death.value_counts(normalize=True))
print(demo_data.non_home_discharge.value_counts(normalize=True))
print(demo_data.anchor_age.describe())
print(demo_data.gender.value_counts(normalize=True))

In [ ]:
ed_comp_data = read_icu_table(core_path_ed, demo_data)

In [ ]:
demo_data.shape, ed_comp_data.shape

#### Process chronic condition history in ED attendances

In [ ]:
diag_data = read_diagnoses(core_path, ed_comp_data)
diag_ext = preproc_icd_module(diag_data, icd_map_path=icd_path, map_code_colname='diagnosis_code', only_icd10=True, cc_dict_path=icd10_map_path)

In [ ]:
diag_ext.subject_id.nunique(), diag_ext.hadm_id.nunique(), ed_comp_data.shape

#### Create patient-level features

In [ ]:
index_features = prepare_unique_patient_features(ed_comp_data, diag_ext, ltc_dict_path=icd10_map_path)

In [ ]:
print(index_features.iloc[2])
print(index_features.iloc[2].ltc_code)

In [ ]:
print(index_features.shape)
print('Unique patients:', index_features.subject_id.nunique())
print('Age distribution:', index_features.anchor_age.describe())
print('Gender distribution:', index_features.gender.value_counts())
print('-------------------------------')
print('Health outcomes')
print(index_features.in_hosp_death.value_counts(normalize=True))
print(index_features.ext_stay_7.value_counts(normalize=True))
print(index_features.non_home_discharge.value_counts(normalize=True))
print(index_features.icu_admission.value_counts(normalize=True))
print('-------------------------------')
print('Comorbidity history')
print(index_features.is_multimorbid.value_counts(normalize=True))
print(index_features.is_complex_multimorbid.value_counts(normalize=True))
print(index_features.phys_men_multimorbidity.value_counts(normalize=True))

#### Test linkage with clinical notes

In [ ]:
notes_data = read_notes_table(core_path_txt, adm_data, ext_path=notes_ext_path)

In [ ]:
notes_data[['subject_id', 'hadm_id', 'text', 'input', 'target', 'input_tokens', 'target_tokens']].to_csv('../outputs/linked_data/linked_notes.csv',
                                                                                                         index=False)

In [ ]:
index_features.shape, index_features.subject_id.nunique(), index_features.hadm_id.nunique()

In [ ]:
index_features.to_csv('../outputs/linked_data/linked_demographics.csv', index=False)

#### Print example notes

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
print('Discharge summary (original)')
pp.pprint(notes_data.text.iloc[14])

In [ ]:
print('-----------------------------------')
print('Discharge summary (preprocessed)')
pp.pprint(notes_data.input.iloc[14])
print('-----------------------------------')
print('Brief Hospital Course segment:')
pp.pprint(notes_data.target.iloc[14])